# Stage of Cell Mitosis Detection - Group Assessment - IDSP1704
#### Scott Donnelly, Louis Chavez, Kyungyoung Her

Mitosis is when a mother cell splits into two genetically similar daughter cells. There are several stages to this:
- Interphase
- Prophase
- Metaphase
- Anaphase
- Telophase
- Cytokensis

The main objective of this project is to be able to identify when cell is in:
- Interphase
- Metaphase
- Anaphase

# Methods:

### compareROI: Compare the nucleus in the video to our reference image to determine mitosis stage
#### Parameters:
- Binary mask of frame from video
- x coordinate of bounding rectangle of nucleus
- y coordinate of bounding rectangle of nucleus
- width of bounding rectangle of nucleus
- height of bounding rectangle of nucleus

#### Algorithm:
- Crop ROI from mask of video
- For each reference image compare the cropped ROI to the shape of the reference image using cv2.matchShapes
- If it's the first image then set that as the closest match and save the index
- If matchShapes returns smaller value then change the value
- Return the index of the closest match

This function is called for each frame of the video

### createRefMasks: Create masks from reference images of the phases to be identifed. Interphase,  Metaphase, Anaphase
#### Parameters:
- None

#### Algorithm:
- Loop through each reference image
- Threshold to get mask
- Morphological closing and opening to fill and holes
- Resize the reference
- Append to list
- Return list of reference images
        
These masks are then used for shape comparison to objects tracked in the input video

# Main Algorithm

1. Process reference images
2. Load the video
3. Identify cells/nucleus using thresholding and contours
4. Box the cells
5. Compare boxed cells to reference image using matchShapes in compareROI
6. Name the stage of mitosis of cell based on the index returned from compareROI
7. Repeat for each frame until finished

In [11]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import image as image

vid = cv2.VideoCapture('mitosiscell360.mp4')
(check, frame) = vid.read()
frame_counter = 0
stage_string = ['Interphase', 'Metaphase', 'Anaphase']

def createRefMasks():
    """
    This function is used to create masks from reference images of the phases to be identifed.
    - Interphase
    - Metaphase
    - Anaphase
        
    These masks are then used for shape comparison to objects tracked in the input video
    """
    ref_strings = ['interphase_ref2_focus.jpg', 'metaphase_ref_focus.jpg', 'anaphase_ref_focus.jpg']
    ref_images = []

    for i in range(0,len(ref_strings)):
        ref_image = cv2.imread(ref_strings[i])
        G = cv2.cvtColor(ref_image, cv2.COLOR_BGR2GRAY)
        ret,thresh = cv2.threshold(G, 140, 235, cv2.THRESH_BINARY_INV)
        shape = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
        CloseRef = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, shape, iterations=2)
        OpenRef = cv2.morphologyEx(CloseRef, cv2.MORPH_OPEN, shape, iterations=2)
        OpenRef = cv2.resize(OpenRef, (100,100))
        ref_images.append(OpenRef)
        #cv2.imshow("ref", ref_images[i])
        cv2.waitKey(0)
        
    return ref_images
        
        
def compareROI(ref_images,mask, x, y, w, h):
    """
    This function uses the OpenCV matchShapes() function to compare the shapes found in the
    video frames with the masks made from reference images. The index returned is to identify the
    labels used on the video.
    """
    match = 0
    for i in range(0,len(ref_images)):
        ROI = cv2.resize(mask[y:y+h, x:x+w], (100,100))
        diff = cv2.matchShapes(ROI, ref_images[i], 1, 0.0)
        if i == 0 or match > diff:
            match = diff
            index = i
            
    return index

ref_images = createRefMasks()

while True:
    
    # Continuous Video Loop
    frame_counter += 1
    if frame_counter == vid.get(cv2.CAP_PROP_FRAME_COUNT):
        frame_counter = 0
        vid.set(cv2.CAP_PROP_POS_FRAMES, 0)
    
    # Thresholding and object detection on the video frames
    ret,thresh = cv2.threshold(frame[:,:,2], 100, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU) # Otsu is auto-global thresholding
    shape = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
    dilated = cv2.dilate(thresh, shape, iterations=2)
    Close = cv2.morphologyEx(dilated, cv2.MORPH_CLOSE, shape)
    contours, _ = cv2.findContours(Close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Loop to detect shapes and draw the rectangles and label the phases for each frame.
    for c in contours:
        if cv2.contourArea(c) > 1000:
            # Epsilon is the maximum distance from contour to approx contour
            # Epsilon = error_rate * actual_arc_length
            epsilon = 0.01 * cv2.arcLength(c, True)
            approx = cv2.approxPolyDP(c, epsilon, True)
            
            # PolyDP provides accurate contouring and object detection.
            #frame = cv2.drawContours(frame, [approx], 0, (0,255,0), 1)

            rect = max(contours,key = cv2.contourArea)
            x,y,w,h = cv2.boundingRect(c)
            stage = compareROI(ref_images, Close, x, y, w, h)
            cv2.putText(frame, stage_string[stage], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (250,120,0), 2)
            frame = cv2.rectangle(frame,(x,y),(x+w,y+h),(250,120,0),2)
    
    #cv2.imshow("Masking", Close)
    cv2.imshow("Video - press \"q\" to stop an any key to close", frame)
    # Slows the frame rate.
    cv2.waitKey(50)
    
    # This delays while waiting for a key to be pressed
    key = cv2.waitKey(50)

    # If the 'q' key is pressed, quit:
    if key == ord("q"):
         break
            
    # Next Frame:
    (check, frame) = vid.read()

    
#Release video and close windows after the loop ends.    
vid.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

# Performance Analysis:

The results of our program show that we are able to identify two out of three of our main goal. We are able to successfully identify when the cells are in interphase and metaphase for most of the time. Our design however is flawed in that it relies on the quality of the reference image and cannot take into account when the the chromosomes split during anaphase as when this occurs our program identifies two new blobs which are generally identical to interphase. To possibly fix this we need to find a way to detect nucleus more accurately or find a way to track the actual cell so that we can identify if there more than one nuclei in the cell
